In [1]:
import pandas as pd
data_path = "../data/open_source_8454_combine_short_description.csv"  # 替換為你的檔案路徑
df = pd.read_csv(data_path)
df_label_0 = df[df["Label"] == 0].sample(n=3000, random_state=42)  # 隨機選擇 3000 筆
df = df.drop(df_label_0.index)  # 從原 DataFrame 刪除這些行
df

,Inbound Message,Label
1,outlook hello team meetings skype meetings etc...,0
4,skype error skype error,0
6,event critical hostname company com value moun...,1
10,engineering tool says connected unable submit ...,0
12,unable login tool sgxqsuojr xwbesorf cards una...,0
...,...,...
8447,erp two accounts added sorry another two accou...,2
8448,tablet needs reimaged due multiple issues crm ...,23
8449,emails coming mail good afternoon receiving em...,22
8452,machine est funcionando unable access machine ...,44


# 十折驗證法

In [2]:
import torch
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"Using device: {device}") 
print(f"CUDA Available: {torch.cuda.is_available()}")
print(f"CUDA Device: {torch.cuda.get_device_name(0)}")
print(f"Current GPU Memory Allocated: {torch.cuda.memory_allocated() / 1e9} GB")
print(f"Current GPU Memory Cached: {torch.cuda.memory_reserved() / 1e9} GB")

Using device: cuda
CUDA Available: True
CUDA Device: NVIDIA GeForce RTX 3050 6GB Laptop GPU
Current GPU Memory Allocated: 0.0 GB
Current GPU Memory Cached: 0.0 GB


In [3]:
from transformers import BertTokenizer
from sklearn.model_selection import StratifiedKFold
# 設定參數
MAX_LENGTH = 128
BATCH_SIZE = 16
EPOCHS = 20

# 初始化 BERT 的 tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# 初始化數據
messages = df['Inbound Message'].tolist()
labels = df['Label'].tolist()

# KFold 初始化
kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

In [4]:
from torch.utils.data import Dataset
class CustomDataset(Dataset):
    def __init__(self, messages, labels, tokenizer, max_length):
        self.messages = messages
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.messages)

    def __getitem__(self, idx):
        message = str(self.messages[idx])
        label = self.labels[idx]
        
        encoding = self.tokenizer.encode_plus(
            message,
            max_length=self.max_length,
            add_special_tokens=True,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [5]:
# 定義訓練與評估函數
def train_epoch(model, dataloader, optimizer, criterion):
    model.train()
    total_loss = 0

    for batch in tqdm(dataloader):
        optimizer.zero_grad()

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()
    
    return total_loss / len(dataloader)

In [6]:
from sklearn.metrics import precision_score, recall_score, f1_score, balanced_accuracy_score, roc_auc_score
from sklearn.preprocessing import label_binarize
import numpy as np
import torch

def eval_model(model, dataloader, criterion, num_labels):
    model.eval()
    total_loss = 0
    all_labels = []
    all_preds = []
    all_probs = []
    
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            loss = criterion(logits, labels)
            total_loss += loss.item()

            probs = torch.softmax(logits, dim=1)
            _, preds = torch.max(logits, dim=1)

            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    val_loss = total_loss / len(dataloader)
    val_accuracy = (np.array(all_preds) == np.array(all_labels)).mean()
    precision = precision_score(all_labels, all_preds, average='weighted', zero_division=0)
    recall = recall_score(all_labels, all_preds, average='weighted', zero_division=0)
    f1 = f1_score(all_labels, all_preds, average='weighted', zero_division=0)
    macro_f1 = f1_score(all_labels, all_preds, average='macro', zero_division=0)
    balanced_acc = balanced_accuracy_score(all_labels, all_preds)

    # **檢查驗證集是否包含兩個以上的類別**
    unique_labels = np.unique(all_labels)
    if len(unique_labels) > 1:
        all_labels_binarized = label_binarize(all_labels, classes=np.arange(num_labels))
        try:
            macro_auc = roc_auc_score(all_labels_binarized, all_probs, multi_class='ovo', average='macro')
            weighted_auc = roc_auc_score(all_labels_binarized, all_probs, multi_class='ovo', average='weighted')
        except ValueError as e:
            print(f"ROC AUC 計算失敗: {e}")
            macro_auc = None
            weighted_auc = None
    else:
        print(f"跳過 ROC AUC 計算，因為驗證集中只有一個類別: {unique_labels}")
        macro_auc = None
        weighted_auc = None

    return {
        "val_loss": val_loss,
        "val_accuracy": val_accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "macro_f1": macro_f1,
        "balanced_accuracy": balanced_acc,
        "macro_auc": macro_auc,
        "weighted_auc": weighted_auc
    }


In [7]:
from sklearn.model_selection import StratifiedKFold
from transformers import BertForSequenceClassification
from torch.utils.data import DataLoader
import torch
from torch.nn import CrossEntropyLoss
from torch.optim import AdamW
from tqdm import tqdm
import numpy as np

# 進行 10 折交叉驗證
fold_results = []

for fold, (train_idx, val_idx) in enumerate(kf.split(messages, labels)):  # 正確用法
    print(f"Fold {fold + 1}")

    train_messages = [messages[i] for i in train_idx]
    train_labels = [labels[i] for i in train_idx]
    val_messages = [messages[i] for i in val_idx]
    val_labels = [labels[i] for i in val_idx]

    # 確保驗證集至少有兩個不同的類別，否則跳過
    unique_val_labels = set(val_labels)
    if len(unique_val_labels) < 2:
        print(f"跳過 Fold {fold + 1}，因為驗證集只有一個類別: {unique_val_labels}")
        continue

    train_dataset = CustomDataset(
        messages=train_messages,
        labels=train_labels,
        tokenizer=tokenizer,
        max_length=MAX_LENGTH
    )
    val_dataset = CustomDataset(
        messages=val_messages,
        labels=val_labels,
        tokenizer=tokenizer,
        max_length=MAX_LENGTH
    )
    train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

    NUM_LABELS = len(set(train_labels) | set(val_labels))  # 確保計算正確
    model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=NUM_LABELS)
    model.to(device)

    optimizer = AdamW(model.parameters(), lr=5e-5, weight_decay=1e-2)
    criterion = CrossEntropyLoss()

    patience = 3
    best_val_loss = float('inf')
    best_metrics = None
    epochs_without_improvement = 0
    
    for epoch in range(EPOCHS):
        print(f"Epoch {epoch + 1}/{EPOCHS}")
        train_loss = train_epoch(model, train_dataloader, optimizer, criterion)
        metrics = eval_model(model, val_dataloader, criterion, num_labels=NUM_LABELS)
        print(metrics)
        
        if metrics['val_loss'] < best_val_loss:
            best_val_loss = metrics['val_loss']
            best_metrics = metrics
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1
        
        if epochs_without_improvement >= patience:
            print(f"Early stopping triggered at epoch {epoch + 1}")
            break
    
    fold_results.append(best_metrics)

# 確保 fold_results 不為空
if len(fold_results) > 0:
    avg_metrics = {metric: np.mean([result[metric] for result in fold_results if result[metric] is not None]) for metric in fold_results[0].keys()}
    print("10-fold Cross-Validation Results:")
    for metric, value in avg_metrics.items():
        print(f"{metric}: {value}")
else:
    print("交叉驗證未成功執行，fold_results 為空")


D:\Anaconda3\envs\cuda\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 7 members, which is less than n_splits=10.
  warnings.warn(


Fold 1


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/20


100%|██████████| 307/307 [02:36<00:00,  1.96it/s]


ROC AUC 計算失敗: Only one class present in y_true. ROC AUC score is not defined in that case.
{'val_loss': 2.1232516578265597, 'val_accuracy': 0.4194139194139194, 'precision': 0.2878949805763176, 'recall': 0.4194139194139194, 'f1': 0.3135941106669882, 'macro_f1': 0.10593165794854827, 'balanced_accuracy': 0.12668280581721822, 'macro_auc': None, 'weighted_auc': None}
Epoch 2/20


100%|██████████| 307/307 [02:35<00:00,  1.97it/s]


ROC AUC 計算失敗: Only one class present in y_true. ROC AUC score is not defined in that case.
{'val_loss': 1.7615428728716713, 'val_accuracy': 0.5183150183150184, 'precision': 0.48376202393514584, 'recall': 0.5183150183150184, 'f1': 0.45384125872906617, 'macro_f1': 0.2278213082871698, 'balanced_accuracy': 0.24665152229895368, 'macro_auc': None, 'weighted_auc': None}
Epoch 3/20


 78%|███████▊  | 239/307 [02:02<00:34,  1.95it/s]


KeyboardInterrupt: 